# Segmentation Algorithm Workflows

This notebook demonstrates IceFloeTracker.jl's segmentation algorithms 
with very basic examples of their use.

In [ ]:
# The notebook presently uses test images and some testing tools from the test directory.
HOME = "../.." # path to the root of the project two levels up

# Activate the environment
using Pkg
Pkg.activate(HOME)
Pkg.precompile()

In [ ]:
# Load packages
using IceFloeTracker: LopezAcosta2019, LopezAcosta2019Tiling, Watkins2025GitHub
using Images: segment_mean, labels_map, SegmentedImage, RGB

## Load the images

Load the dataset from https://github.com/danielmwatkins/ice_floe_validation_dataset using the `Watkins2025GitHub` data loader.

In [ ]:
data_loader = Watkins2025GitHub(; ref="a451cd5e62a10309a9640fbbe6b32a236fcebc70")

The available data are listed in the metadata field:

In [ ]:
data_loader().metadata

For the example, we choose a single case from Baffin Bay in May 2022.

In [ ]:
dataset = data_loader(c-> c.case_number == 6 && c.satellite == "terra")
case = first(dataset)

The data include the true-color image:

In [ ]:
truecolor = RGB.(case.modis_truecolor) # TODO: remove RGB cast

... a false-color image:

In [ ]:
falsecolor = RGB.(case.modis_falsecolor) # TODO: remove RGB cast

... and a landmask, which in this particular case is empty:

In [ ]:
landmask = RGB.(case.modis_landmask) # TODO: remove RGB cast

## Run the segmentation algorithm

The segmentation algorithm is an object with parameters as follows:

In [ ]:
segmentation_algorithm = LopezAcosta2019()

If we wanted to modify the options, we could include those in the call above. 
See the documentation for `LopezAcosta2019` for details.
The default parameters are as follows:

In [50]:
dump(segmentation_algorithm)

LopezAcosta2019
  landmask_structuring_element: Array{Bool}((99, 99)) Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]


Run the algorithm as follows:

In [ ]:
segments = segmentation_algorithm(truecolor, falsecolor, landmask)

To show the results with each segment marked using its mean color:

In [ ]:
map(i -> segment_mean(segments, i), labels_map(segments))

We can do the same with the falsecolor image:

In [ ]:
# Get the labels_map
segments_falsecolor = SegmentedImage(falsecolor, labels_map(segments))
map(i -> segment_mean(segments_falsecolor, i), labels_map(segments_falsecolor))

## Run the segmentation algorithm with tiling
The "tiling" version of the algorithm is an object:

In [ ]:
segmentation_algorithm_with_tiling = LopezAcosta2019Tiling()

It has more configurable parameters. 
For details, see the documentation of LopezAcosta2019Tiling.
The default parameters are as follows:

In [54]:
dump(segmentation_algorithm_with_tiling)

LopezAcosta2019Tiling
  tile_settings: @NamedTuple{rblocks::Int64, cblocks::Int64}
    rblocks: Int64 2
    cblocks: Int64 2
  cloud_mask_thresholds: @NamedTuple{prelim_threshold::Float64, band_7_threshold::Float64, band_2_threshold::Float64, ratio_lower::Float64, ratio_upper::Float64, r_offset::Float64}
    prelim_threshold: Float64 0.43137254901960786
    band_7_threshold: Float64 0.7843137254901961
    band_2_threshold: Float64 0.7450980392156863
    ratio_lower: Float64 0.0
    ratio_upper: Float64 0.75
    r_offset: Float64 0.0
  adapthisteq_params: @NamedTuple{white_threshold::Float64, entropy_threshold::Int64, white_fraction_threshold::Float64}
    white_threshold: Float64 25.5
    entropy_threshold: Int64 4
    white_fraction_threshold: Float64 0.4
  adjust_gamma_params: @NamedTuple{gamma::Float64, gamma_factor::Float64, gamma_threshold::Int64}
    gamma: Float64 1.5
    gamma_factor: Float64 1.3
    gamma_threshold: Int64 220
  structuring_elements: @NamedTuple{se_disk1::Matri

In [ ]:
display("text/markdown", @doc LopezAcosta2019Tiling)

In [ ]:
segments = segmentation_algorithm_with_tiling(truecolor, falsecolor, landmask)

To show the results with each segment marked using its mean color:

In [ ]:
map(i -> segment_mean(segments, i), labels_map(segments))

With the falsecolor image:

In [ ]:
# Get the labels_map
segments_falsecolor = SegmentedImage(falsecolor, labels_map(segments))
map(i -> segment_mean(segments_falsecolor, i), labels_map(segments_falsecolor))